<a href="https://colab.research.google.com/github/B-Lorentz/physics-bsc-visual/blob/main/15_Quantum_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from jax import numpy as jnp
from jax import grad, vmap, jit, random, lax, ops
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import interact, widgets

In [7]:
x1 = x2 = np.linspace(0, 1, 100)

def ploti(name, f):
  x1s, x2s = np.meshgrid(x1, x2)
  plt.contourf(x1, x2, f(x1s, x2s), levels=50,cmap=plt.get_cmap("coolwarm"))
  plt.plot([0,1],[0,1], "black")
  #plt.plot(x1, psi1(x1))
  plt.xlabel("$x_1$", fontsize=14)
  plt.xlabel("$x_2$", fontsize=14)
  plt.title(name, fontsize=16)
def interplot(n1, n2):
  psi1 = lambda x: jnp.sin(n1*np.pi*x)
  psi2 = lambda x: jnp.sin(n2*np.pi*x)
  plt.figure(figsize=(16, 8))
  plt.subplot(121)
  ploti("Bozon", lambda x1, x2: psi1(x1)*psi2(x2) + psi1(x2)*psi2(x1))

  plt.subplot(122)
  ploti("Fermion", lambda x1, x2: psi1(x1)*psi2(x2) - psi1(x2)*psi2(x1))
interact(interplot, n1=widgets.IntSlider(1, 1, 10), n2=widgets.IntSlider(1, 3, 10))


interactive(children=(IntSlider(value=1, description='n1', max=10, min=1), IntSlider(value=3, description='n2'…

<function __main__.interplot>

In [9]:
def MCn(Es, beta, mu, stat):
  N_es = Es.shape[0]
  efun = lambda ns: (ns*(Es-mu)).sum()
  def mcf(key):
    def upd(i, state):
      ns, sum_ns, key = state
      k1, k2, k3, k4 = random.split(key, 4)
      inds = random.choice(k1, N_es, shape=(N_es//20,), replace=False)
      ns2 = ns.at[inds].set(stat.proposer(k2, ns[inds]))
      e0 = efun(ns)
      e1 = efun(ns2)
      dec = jnp.logical_or(e1<e0, jnp.exp(-beta*(e1-e0)) > random.uniform(k3, (1,)))
      ns = lax.cond(dec[0], lambda x:ns2, lambda x:x, ns)
      sum_ns = lax.cond(i%5 == 4, lambda x: x+ns, lambda x:x, sum_ns)
      return (ns, sum_ns, k4)
    k1, k2 = random.split(key)
    init = (stat.proposer(k1, np.zeros(N_es, dtype=int)), np.zeros(N_es, dtype=int), k2)
    res = lax.fori_loop(0, 1000, upd, init)[1]
    return res
  return mcf
class Stat:
  def __init__(self, name, f, proposer):
    self.name = name
    self.f = f
    self.proposer = proposer
  
fermi = Stat("F", lambda E, beta, mu : 1/(np.exp(beta*(E-mu))+1), lambda key, ns: random.choice(key, 2, shape=ns.shape ))

In [10]:
N_stat = 100
Es = np.linspace(0, 1.0, N_stat)

def plotbuilder(stat):
  def plotter(beta, mu):
    plt.figure(figsize=(10, 8))
    ns = stat.f(Es, beta, mu)
    plt.plot(Es, ns, label=f"$f_{stat.name}(E)$")
    bns = np.exp(-beta*Es)
    plt.plot(Es, bns*ns.max(), label="Boltzmann")
    plt.plot([mu, mu], [0, ns.max()], "r", label=r"$\mu$")
    plt.xlabel("E", fontsize=14)
    plt.ylabel("<n>", fontsize=14)
    plt.grid()
    mcs = vmap(MCn(Es, beta, mu, stat), 0)(random.split(random.PRNGKey(127), 100))
    res, resd = mcs.mean(0), mcs.std(0)
    maxi = res.max()/ns.max()
    #print(res.shape, res.min(), res.max())
    plt.errorbar(Es, res/maxi, resd/maxi)
    plt.title(r"$\beta =" +f"{beta:.2f}$")
  
    plt.legend(fontsize=14)
  return plotter
  
interact(plotbuilder(fermi), beta=widgets.FloatSlider(min=1.0, max=30, value=3.5), mu=widgets.FloatSlider(min=-1, max=1, value=0.1, step=0.05))
None


interactive(children=(FloatSlider(value=3.5, description='beta', max=30.0, min=1.0), FloatSlider(value=0.1, de…

In [12]:
bose = Stat("B", lambda E, beta, mu : 1/(np.exp(beta*(E-mu))-1), lambda key, ns: jnp.maximum((random.normal(key, ns.shape)*jnp.maximum(ns, 10)/2).astype(int)+ns, 0) )
interact(plotbuilder(bose), beta=widgets.FloatSlider(min=1.0, max=10, value=1.5), mu=widgets.FloatSlider(min=-1, max=-0.15, value=-0.05, step=0.05))
None

interactive(children=(FloatSlider(value=1.5, description='beta', max=10.0, min=1.0), FloatSlider(value=-0.15, …